In [ ]:
!pip install mlflow dagshub --quiet

In [ ]:
import os
import mlflow
from mlflow.tracking import MlflowClient
import dagshub
from google.colab import userdata
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import matplotlib.font_manager as fm

In [ ]:
font_path = os.path.expanduser("/content/drive/MyDrive/Colab Notebooks/fonts/Exo2-VariableFont_wght.ttf")  # Remplacez par le chemin exact
fm.fontManager.addfont(font_path)

# Définir la police globale avec le nom de la police
rcParams["font.family"] = "Exo 2"
# deux couleurs pertinentes pour aller avec la présentation
bleuclair = (0.15, 0.55, 0.82)
couleur_complementaire = (1 - bleuclair[0], 1 - bleuclair[1], 1 - bleuclair[2])
bleufonce = "#073642"

imgPrezPath = '/content/drive/MyDrive/Colab Notebooks/Projet 8/img'

In [ ]:
# Récupère automatiquement le secret
dagshub_token = userdata.get('DAGSHUB_TOKEN')

# Initialisation Dagshub
dagshub.auth.add_app_token(dagshub_token)

# Connecter MLflow à Dagshub
dagshub.init(repo_owner='fabiencappelli', repo_name='Projet_08', mlflow=True)

# Configure MLflow pour pointer vers Dagshub
mlflow.set_tracking_uri('https://dagshub.com/fabiencappelli/Projet_08.mlflow')

baseline sans aug
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/41/runs/94a9f8dd4e744b5591e98e0bac2b3259

baseline avec aug
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/42/runs/f584368d4f30439fad67f925c387071c

Full Unet Resnet sans aug
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/43/runs/01916c7e305f44b4b28a401c995e75d4

Full Unet Resnet
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/37/runs/8f64b5db2257437191ca5dd2afa925c4

Full Linknet efficientnetb0
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/38/runs/e060cfebab244676b482cf57524fe91b

Full FPN efficientnetb0
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/39/runs/09a7caf9d4cb4175b22813e115c6ccd7

Full FPN efficientnetb0 w/ Dice_Loss
https://dagshub.com/fabiencappelli/Projet_08.mlflow/#/experiments/40/runs/b4765d19646641b48ed9fa8df6c2ec42

In [ ]:
noms = [
    "Baseline ss. aug.",
    "Baseline av. aug.",
    "Full Unet Resnet ss. aug.",
    "Full Unet Resnet",
    "Full Linknet efficientnetb0",
    "Full FPN efficientnetb0",
    "Full FPN efficientnetb0 av. Dice_Loss"
]

run_ids = [
    "94a9f8dd4e744b5591e98e0bac2b3259",
    "f584368d4f30439fad67f925c387071c",
    "01916c7e305f44b4b28a401c995e75d4",
    "8f64b5db2257437191ca5dd2afa925c4",
    "e060cfebab244676b482cf57524fe91b",
    "09a7caf9d4cb4175b22813e115c6ccd7",
    "b4765d19646641b48ed9fa8df6c2ec42"
]

df = pd.DataFrame({"nom": noms, "run_id": run_ids})
print(df)

In [ ]:
# Liste des métriques à rapatrier
metrics_wanted = ['iou_score', 'test_avg_inference_time', 'test_mean_iou']

In [ ]:
# Pour chaque run, on ajoute les métriques demandées
for metric in metrics_wanted:
    df[metric] = None  # On crée la colonne vide

for idx, row in df.iterrows():
    run_id = row['run_id']
    run = mlflow.get_run(run_id)
    for metric in metrics_wanted:
        value = run.data.metrics.get(metric)
        df.at[idx, metric] = value

In [ ]:
print(df)

In [ ]:
palette = sns.color_palette("Set2", 4)
sns.set(style='whitegrid', palette=palette)

fig, ax1 = plt.subplots(figsize=(12, 8))


ln_test, = ax1.plot(df['nom'], df['test_mean_iou'], label='IoU Test', marker='o', linewidth=2, color=palette[0])
ln_train, = ax1.plot(df['nom'], df['iou_score'], label='IoU Train', marker='s', linewidth=2, color=palette[1])

ax1.set_xlabel('Modèles')
ax1.set_ylabel('Scores')
ax1.tick_params(axis='y')

# Fusion des légendes
lines = [ln_test, ln_train]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left', frameon=True, fancybox=True, shadow=True)

plt.setp(ax1.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.title('Comparaison des performances des modèles', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "IoU.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)

In [ ]:
palette = sns.color_palette("Set2", 4)
sns.set(style='whitegrid', palette=palette)
plt.figure(figsize=(12, 8))

plt.plot(df['nom'], df['test_mean_iou'], label='IoU Test', marker='o')
plt.plot(df['nom'], df['iou_score'], label='IoU Train', marker='s')

plt.xlabel('Modèles', labelpad=30)
plt.ylabel('Scores', labelpad=30)
plt.title('Comparaison des performances des modèles', fontsize=14, pad=50)
plt.legend(loc='upper left', frameon=True, fancybox=True, shadow=True)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "IoU.svg"), format="svg", bbox_inches="tight", pad_inches=0.1)
plt.show()

In [ ]:
palette = sns.color_palette("Set1", 1)
sns.set(style='whitegrid', palette=palette)
plt.figure(figsize=(12, 8))
plt.plot(df['nom'], df['test_avg_inference_time'], marker='o', label='Temps d\'inférence moyen')

plt.xlabel('Modèles', labelpad=30)
plt.ylabel('Temps d\'inférence (s.)', labelpad=30)
plt.title('Comparaison des temps d\'inférence des modèles', fontsize=14, pad = 50)
plt.legend(loc='upper left', frameon=True, fancybox=True, shadow=True)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "inference_time.svg"))
plt.show()

In [ ]:
/content/drive/MyDrive/Colab Notebooks/Projet 8/FULL_efficientnetb0_FPN_BS_8_LR_0.001_DICE_LOSS.keras

In [ ]:
!pip install segmentation_models

In [ ]:

import os

os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

from tensorflow.keras.activations import swish
from segmentation_models.metrics import IOUScore
from tensorflow import keras

def total_loss(y_true, y_pred):
    return crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# Ajoute toutes les fonctions/objets custom que tu as utilisés
custom_objects = {
    "swish": swish,
    "total_loss" : total_loss
}

model = keras.models.load_model(
    "/content/drive/MyDrive/Colab Notebooks/Projet 8/FULL_efficientnetb0_FPN_BS_8_LR_0.001_DICE_LOSS.keras",
    custom_objects={
        "swish": swish,
        "total_loss": total_loss
    },
    compile=False
)
model.summary()


In [ ]:
total_params = model.count_params()
print("Nombre total de paramètres :", total_params)